# LeetCode Problem Solutions
See [https://leetcode.com/problems/](https://leetcode.com/problems/) for more information. My goal here is to solve the problems efficiently, memory and computation wise. Here are some ideas to achieve efficiency:
* Tuples are immutable and likewise, are more memory efficient than lists.
* Sets are based on a hash table and likewise much faster than lists when searching.

# Brain Teaser Problems

### Car-Pooling
  
https://leetcode.com/problems/car-pooling/  

You are driving a vehicle that has capacity empty seats initially available for passengers.  The vehicle only drives east (ie. it cannot turn around and drive west.)

Given a list of trips, trip[i] = [num_passengers, start_location, end_location] contains information about the i-th trip: the number of passengers that must be picked up, and the locations to pick them up and drop them off.  The locations are given as the number of kilometers due east from your vehicle's initial location.

Return true if and only if it is possible to pick up and drop off all passengers for all the given trips.

**Problem Breakdown:** 
* Order trips by stops, i.e., when the next stop will be.
    * Consider dropoffs to be before pickups with the same timestamp.
* Go through the stops and check if capacity has been reached on each steps.

In [74]:
class Solution(object):
    def carPooling(self, trips, capacity):
        """
        :type trips: List[List[int]]
        :type capacity: int
        :rtype: bool
        """
        actions = [] # [step, n]
        for n, s, e in trips:
            actions.append((s, n))
            actions.append((e, -n))
        actions.sort() # key=lambda x: (x[0], x[1]) is implied
        n = 0
        while actions:
            n += actions.pop(0)[1]
            if n > capacity: return False
        return True

assert(Solution().carPooling([], 0))
assert(Solution().carPooling([[2,1,5],[3,3,7]], 5))
assert(not Solution().carPooling([[2,1,5],[3,3,7]], 4))
assert(Solution().carPooling([[2,1,5],[3,5,7]], 3))

# Singly-Linked List Problems


### Linked List Cycle

https://leetcode.com/problems/linked-list-cycle-ii/

Given a linked list, return the node where the cycle begins. If there is no cycle, return null.

**Problem Breakdown:**
* Rabbit moves 2 positions every step, while turtle moves 1. One of two things will happen:
    * Rabbit will reach the end of the list.
    * Rabbit and turtle will meet somewhere in the cycle.
    * https://math.stackexchange.com/questions/913499/proof-of-floyd-cycle-chasing-tortoise-and-hare
* When the rabbit and turtle meet, they are necessarily *r* steps aways from the start of the cycle.
    * At this point, ignore the rabbit and put another turtle at the head.
    * Since *T = kC + r*, then the two turtles will meet at the start of the cycle.

In [90]:
class Solution(object):
    def detectCycle(self, head):
        """
        :type head: ListNode
        :rtype: ListNode
        """
        t = r = head
        while r and r.next:
            r = r.next.next
            t = t.next 
            if r == t:
                t2 = head
                while True:
                    if t == t2: return t
                    t = t.next
                    t2 = t2.next      
        return None

### Reverse Linked List II

https://leetcode.com/problems/reverse-linked-list-ii/

Reverse a linked list from position m to n. Do it in one-pass.  
Note: 1 ≤ m ≤ n ≤ length of list.


**Problem Breakdown:**  

Suppose the problem is:
> 1 --> 2 --> 3 --> 4 --> 5 --> 6 --> 7  
> Reverse 3 --> 5 so it looks like:  
> 1 --> 2 --> 5 --> 4 --> 3 --> 6 --> 7  

Conceptually, we can break the linked list into three sections: beginning, middle (section we want to reverse), and end. The three parts would look like:  
> 1 --> 2  
> 3 --> 4 --> 5  
> 6 --> 7  

From there, we can get our solution by reversing the middle and then attach the middle's:
* new head (5) to the tail of the beginning list (2).
* new tail (3) to the head of the end list (6).

This is straightforward, but the solution is cumbersome to implement because:
* We are dealing with a singly linked list.
    * We need to keep track of the endpoints of the sub-lists.
* Special logic needed if the head is part of the reversal segment (i.e. *m=1*).
* The reversal algorithm is somewhat complex, see it's [LeetCode problem](https://leetcode.com/problems/reverse-linked-list/).

In [92]:
class Solution(object):
    def reverseBetween(self, head, m, n):
        """
        :type head: ListNode
        :type m: int
        :type n: int
        :rtype: ListNode
        """
        # Base cases
        if not head: return head
        if not head.next: return head
        if m == n: return head
        
        # Define left/cur/right
        self.left = None
        self.cur = head
        self.right = head.next
        
        # Define function for shifting
        def shift():
            self.left = self.cur
            self.cur = self.right
            self.right = self.right.next
        
        # Shift until we reach the section to reverse.
        # M is the tail node of the begininng list.
        # M1 is head node of the 'reversal' section (will end up being tail).
        for _ in range(m-1): shift()
        M = self.left
        M1 = self.cur
        
        # Set M to point to None. We will connect it to the tail
        #   of the 'reversal' section at the end 
        # Note: This is not required, but provides clarity to the solution.
        if m > 1: M.next = None
            
        # Set left to None so reversal algorithm works properly
        self.left = None
        
        # Reveres the m'th to n'th nodes
        for _ in range(n-m):
            self.cur.next = self.left
            shift()
        self.cur.next = self.left
        
        # Connect the reversed section to the first part of the list
        if m > 1: M.next = self.cur
        else: head = self.cur
            
        # Connect he first node of the reversal section to the end of the list
        M1.next = self.right
        
        # Return
        return(head)
       

# Graph Problems

### Is Graph Bipartite

https://leetcode.com/problems/is-graph-bipartite/submissions/

Determine if the nodes of a given graph can be partitioned into two independent sets A and B such that every edge in the graph connects a node in set A and a node in set B (is it bipartite).  

Notes:
* Graph is undirected and has no self-edges.
* Graph may not be connected.

**Problem Breakdown:**  

We approach this problem by 'coloring' the nodes to represent whether they are in set A or B. If a graph is bipartite, then any two connected nodes must be of different colors.  

We use a recursive function which takes a node and a color:
* The node is assigned the color (e.g. blue).
* The function is recalled for each neighboor node and passed in the other color (e.g. red).
* If the node is already colored:
    * Check if it's color matches the passed color.
        * If it doesn't match, then **the graph is not bipartite**.
    * Do not make any more calls on this branch.

An inital call of the recursive function must be called for every node since the graph is not guranteed to be connected.

In [1]:
class Solution(object):
    def isBipartite(self, graph):
        """
        :type graph: List[List[int]]
        :rtype: bool
        """
        def doSection(graph, colors, n, col):
            # Stop if graph already shown to not be bipartite
            if not self.is_bipartite: return
            
            # If node is already colored, check if bipartite and return
            if colors[n]:
                if colors[n] != col: self.is_bipartite = False
                return

            # Color node and decide connection colors
            colors[n] = col
            if col == 'blue': conColor = 'red'
            else: conColor = 'blue'
            
            # Recursive for all connecting nodes
            for con in graph[n]:
                doSection(graph, colors, con, conColor)
            
        # Initialize
        self.is_bipartite = True
        colors = [None]*len(graph)
        
        # Run
        for n in range(len(graph)):
            if not colors[n]: doSection(graph, colors, n, 'blue')
            if not self.is_bipartite: return False
        return True